In [1]:
pip install numpy pandas pillow torch torchvision scikit-learn matplotlib torchsummary torchaudio 


Note: you may need to restart the kernel to use updated packages.


In [2]:
# 3D-Print Defect Detection from CSV — ResNet-9 (scratch)
# Train on images/all_images256, Test on images/test_images_oblique256 + test_images_silver265 (oblique=กล้องรอง,silver=กล้องรอง )
# CSVs: general_data/all_images_no_filter.csv (train), general_data/all_images_no_filter.csv or specific test CSVs if needed
# Works on ROCm/NVIDIA automatically. (AKE=7800xt,i5 13500H)

import os, random, numpy as np, pandas as pd
from pathlib import Path
from PIL import Image
import torch, torch.nn as nn, torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, accuracy_score
import matplotlib.pyplot as plt
from torchvision.ops.misc import Conv2dNormActivation
import pandas as pd
import numpy as np
from pathlib import Path
from PIL import Image
from sklearn.model_selection import GroupShuffleSplit, StratifiedShuffleSplit

# ========= Repro & Device =========
SEED = 1337
random.seed(SEED); np.random.seed(SEED); torch.manual_seed(SEED); torch.cuda.manual_seed_all(SEED)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device:", device)



Device: cuda


In [3]:
import torch
print(torch.__version__)
print(torch.version.hip)
print(torch.cuda.is_available())
print(torch.cuda.get_device_name(0))


2.10.0.dev20250925+rocm6.4
6.4.43484-123eb5128
True
AMD Radeon RX 7800 XT


In [4]:
# --- พารามิเตอร์ที่ต้องตั้งให้ตรงโปรเจกต์ ---
ROOT         = Path("/home/ake/envs/rocm_env/bin/CNN/Printing_Errors")
IMAGES_ROOT  = ROOT / "images"              # โฟลเดอร์ภาพหลัก
TRAIN_SUBDIR = "all_images256"              # โฟลเดอร์ย่อยที่เก็บภาพ train
CSV_MASTER   = ROOT / "general_data" / "all_images_no_filter.csv"  # CSV หลัก
SPLIT_DIR    = ROOT / "splits"              # โฟลเดอร์สำหรับบันทึกผล split
CLASS_RAW    = [0,1,2,4]                    # คลาสที่ต้องการใช้

SPLIT_DIR.mkdir(parents=True, exist_ok=True)

# --- โหลด CSV (ทน delimiter) ---
df_master = pd.read_csv(CSV_MASTER, sep=None, engine='python', encoding='utf-8')
df_master.columns = [c.strip() for c in df_master.columns]
assert {"image","class"}.issubset(df_master.columns), "CSV must include 'image' and 'class'"

# --- คัดเฉพาะคลาสเป้าหมาย ---
df_master = df_master[df_master["class"].isin(CLASS_RAW)].copy()

# --- ทำให้ path เป็น string/normalize ---
df_master["image"] = (
    df_master["image"].astype(str).str.strip().str.replace("\\", "/", regex=False)
)

# --- เติมโฟลเดอร์ย่อย ถ้าไม่มี prefix โฟลเดอร์ ---
def ensure_subdir_path(s: str) -> str:
    return s if "/" in s else f"{TRAIN_SUBDIR}/{s}"
df_master["image"] = df_master["image"].apply(ensure_subdir_path)

# --- เก็บเฉพาะภาพที่อยู่ในโดเมน train_subdir ---
df_master = df_master[df_master["image"].str.startswith(f"{TRAIN_SUBDIR}/", na=False)].reset_index(drop=True)

# --- เก็บเฉพาะไฟล์ที่มีอยู่จริง ---
def exists_image(rel_path: str) -> bool:
    return (Path(IMAGES_ROOT) / rel_path).exists()
df_master = df_master[df_master["image"].apply(exists_image)].reset_index(drop=True)

print("Total images (train domain & exists):", len(df_master))

# --- แบ่ง Train/Test เท่านั้น: train ~70% / test ~30% ---
def split_train_test(df, group_col="recording", test_size=0.30, seed=42):
    """แบ่ง Train/Test เท่านั้น (ไม่มี validation)"""
    if group_col in df.columns and df[group_col].notna().any():
        groups = df[group_col].fillna("nogroup")
        gss = GroupShuffleSplit(n_splits=1, test_size=test_size, random_state=seed)
        i_tr, i_te = next(gss.split(df, groups=groups))
        return df.iloc[i_tr].copy(), df.iloc[i_te].copy()
    else:
        y = df["class"]
        sss = StratifiedShuffleSplit(n_splits=1, test_size=test_size, random_state=seed)
        i_tr, i_te = next(sss.split(df, y))
        return df.iloc[i_tr].copy(), df.iloc[i_te].copy()

# --- ทำการ split และเซฟไฟล์ ---
df_tr, df_te = split_train_test(df_master, group_col="recording", test_size=0.30, seed=42)

df_tr.to_csv(SPLIT_DIR/"train.csv", index=False)
df_te.to_csv(SPLIT_DIR/"test.csv",  index=False)

print("Split sizes -> Train:", len(df_tr), " Test:", len(df_te))
print("\nTrain class counts:\n", df_tr["class"].value_counts().sort_index())
print("\nTest class counts:\n",  df_te["class"].value_counts().sort_index())

Total images (train domain & exists): 16287
Split sizes -> Train: 11456  Test: 4831

Train class counts:
 class
0    5133
1    3184
2    2977
4     162
Name: count, dtype: int64

Test class counts:
 class
0    1893
1    2328
2     532
4      78
Name: count, dtype: int64


In [5]:
# === Load splits (val เป็นออปชัน) ===
from pathlib import Path
import pandas as pd
from IPython.display import display


# โหลด train/test (บังคับมี)
df_tr = pd.read_csv(SPLIT_DIR/"train.csv")
df_te = pd.read_csv(SPLIT_DIR/"test.csv")

# โหลด val ถ้ามี
val_path = SPLIT_DIR/"val.csv"
df_va = pd.read_csv(val_path) if val_path.exists() else None

print("Shapes | Train:", df_tr.shape, "| Test:", df_te.shape, "| Val:", (df_va.shape if df_va is not None else None))

# --- ฟังก์ชันช่วย ---
def count_classes(df: pd.DataFrame):
    vc = df["class"].value_counts().sort_index()
    return vc.to_dict()

def exists_rate(df: pd.DataFrame, root: Path):
    paths = [(root / p) for p in df["image"].astype(str)]
    n = len(paths)
    n_ok = sum(p.exists() for p in paths)
    return n_ok, n, (n_ok / n if n else 0.0)

def sample_paths(df: pd.DataFrame, k=3):
    k = min(k, len(df))
    if k == 0: return []
    return df.sample(k, random_state=1337)["image"].astype(str).tolist()

# --- นับคลาส ---
print("Class counts (train):", count_classes(df_tr))
if df_va is not None:
    print("Class counts (val)  :", count_classes(df_va))
print("Class counts (test) :", count_classes(df_te))

# --- ตรวจไฟล์มีจริงในดิสก์ ---
ok_tr, n_tr, r_tr = exists_rate(df_tr, IMAGES_ROOT)
if df_va is not None:
    ok_va, n_va, r_va = exists_rate(df_va, IMAGES_ROOT)
ok_te, n_te, r_te = exists_rate(df_te, IMAGES_ROOT)

msg_val = f", Val: {ok_va}/{n_va} ({r_va:.2%})" if df_va is not None else ""
print(f"Exists rate -> Train: {ok_tr}/{n_tr} ({r_tr:.2%}){msg_val}, Test: {ok_te}/{n_te} ({r_te:.2%})")

# --- โชว์ head ---
print("\n[Train head]")
display(df_tr.head(3))
if df_va is not None:
    print("\n[Val head]")
    display(df_va.head(3))
print("\n[Test head]")
display(df_te.head(3))

# --- ตัวอย่างพาธและสถานะไฟล์ ---
for name, df in [("Train", df_tr), ("Val", df_va), ("Test", df_te)]:
    if df is None:
        print(f"\n[{name} samples] (skipped: no file)")
        continue
    print(f"\n[{name} samples]")
    for rel in sample_paths(df, k=3):
        p = IMAGES_ROOT / rel
        print(" -", rel, "| exists:", p.exists())


Shapes | Train: (11456, 17) | Test: (4831, 17) | Val: None
Class counts (train): {0: 5133, 1: 3184, 2: 2977, 4: 162}
Class counts (test) : {0: 1893, 1: 2328, 2: 532, 4: 78}
Exists rate -> Train: 11456/11456 (100.00%), Test: 4831/4831 (100.00%)

[Train head]


,image,class,layer,nozzle,filament,ex_mul,retraction,layer_height,filament_color,shape,recording,printbed_color,extrusion_multiplier,extrusion_std,modified_layers,brightness,mean_brightness
0,all_images256/ELP_12MP_01.12.2022_166990882982...,0,4,0.4,PLA,1.05,8.0,0.3,gray,1,Recording_01122022_15_27_47,black,NaN,NaN,NaN,164,145
1,all_images256/ELP_12MP_01.12.2022_166990882986...,0,4,0.4,PLA,1.05,8.0,0.3,gray,1,Recording_01122022_15_27_47,black,NaN,NaN,NaN,157,145
2,all_images256/ELP_12MP_01.12.2022_166990887746...,0,5,0.4,PLA,1.05,8.0,0.3,gray,1,Recording_01122022_15_27_47,black,NaN,NaN,NaN,150,145



[Test head]


,image,class,layer,nozzle,filament,ex_mul,retraction,layer_height,filament_color,shape,recording,printbed_color,extrusion_multiplier,extrusion_std,modified_layers,brightness,mean_brightness
0,all_images256/ELP_12MP_01.12.2022_166992843887...,0,4,0.4,PLA,1.05,8.0,0.3,gray,3,Recording_01122022_17_44_58,black,NaN,NaN,NaN,153,137
1,all_images256/ELP_12MP_01.12.2022_166992843891...,0,4,0.4,PLA,1.05,8.0,0.3,gray,3,Recording_01122022_17_44_58,black,NaN,NaN,NaN,147,137
2,all_images256/ELP_12MP_01.12.2022_166992846382...,0,5,0.4,PLA,1.05,8.0,0.3,gray,3,Recording_01122022_17_44_58,black,NaN,NaN,NaN,152,137



[Train samples]
 - all_images256/ELP_12MP_14.02.2023_167639778843.png | exists: True
 - all_images256/ELP_12MP_21.12.2022_167165785322.png | exists: True
 - all_images256/ELP_12MP_17.02.2023_167664745761.png | exists: True

[Val samples] (skipped: no file)

[Test samples]
 - all_images256/ELP_12MP_04.01.2023_167284876744.png | exists: True
 - all_images256/ELP_12MP_23.02.2023_167716438692.png | exists: True
 - all_images256/ELP_12MP_06.12.2022_167036023452.png | exists: True


In [6]:
# mapping raw class id -> label name
CLASS_NAMES = {
    0: "Good",
    1: "Under-Extrusion",
    2: "Stringing",
    4: "Spaghetti"
}

print("Train class counts:\n", df_tr["class"].map(CLASS_NAMES).value_counts())
print("Test class counts:\n",  df_te["class"].map(CLASS_NAMES).value_counts())


Train class counts:
 class
Good               5133
Under-Extrusion    3184
Stringing          2977
Spaghetti           162
Name: count, dtype: int64
Test class counts:
 class
Under-Extrusion    2328
Good               1893
Stringing           532
Spaghetti            78
Name: count, dtype: int64


In [7]:
# --- Custom Dataset ---
class CustomImageDataset(Dataset):
    def __init__(self, df, root_dir, transform=None):
        self.df = df.reset_index(drop=True)
        self.root_dir = Path(root_dir)
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        img_path = self.root_dir / self.df.loc[idx, "image"]
        label = int(self.df.loc[idx, "class"])
        image = Image.open(img_path).convert("RGB")
        if self.transform:
            image = self.transform(image)
        return image, label

# --- Transform ---
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    # transforms.Normalize(...)  # ใส่ถ้าคุณมี mean/std
])

# --- Dataset ---
ds_train = CustomImageDataset(df_tr, IMAGES_ROOT, transform=transform)
ds_test  = CustomImageDataset(df_te, IMAGES_ROOT, transform=transform)

# --- DataLoader ---
BATCH = 8
kwargs = dict(batch_size=BATCH, num_workers=0, pin_memory=torch.cuda.is_available())

train_ld = DataLoader(ds_train, shuffle=True,  **kwargs)
test_ld  = DataLoader(ds_test,  shuffle=False, **kwargs)

In [8]:
num_classes = 4 
# ========= Model: ResNet-9 =========
def conv_bn_relu(in_c, out_c, k=3, s=1, p=1):
    return Conv2dNormActivation(in_c, out_c, kernel_size=k, stride=s, padding=p,
                                norm_layer=nn.BatchNorm2d, activation_layer=nn.ReLU)

class BasicBlock(nn.Module):
    def __init__(self, c):
        super().__init__()
        self.conv1 = conv_bn_relu(c, c)
        self.conv2 = Conv2dNormActivation(c, c, kernel_size=3, padding=1,
                                          norm_layer=nn.BatchNorm2d, activation_layer=None)
        self.relu = nn.ReLU(inplace=True)
    def forward(self, x):
        id = x
        x = self.conv1(x)
        x = self.conv2(x)
        x = x + id
        return self.relu(x)

class ResNet9(nn.Module):
    def __init__(self, in_ch=3, num_classes=4):
        super().__init__()
        self.layer1 = conv_bn_relu(in_ch, 64, 3, 1, 1)
        self.layer2 = conv_bn_relu(64, 128, 3, 2, 1)  # /2
        self.res1   = BasicBlock(128)
        self.layer3 = conv_bn_relu(128, 256, 3, 2, 1) # /4
        self.layer4 = conv_bn_relu(256, 512, 3, 2, 1) # /8
        self.res2   = BasicBlock(512)
        self.pool   = nn.AdaptiveAvgPool2d(1)
        self.fc     = nn.Linear(512, num_classes)
    def forward(self, x):
        x = self.layer1(x)
        x = self.layer2(x); x = self.res1(x)
        x = self.layer3(x)
        x = self.layer4(x); x = self.res2(x)
        x = self.pool(x).flatten(1)
        return self.fc(x)

model = ResNet9(num_classes=num_classes).to(device)


In [9]:
# ตรวจสอบ model summary
from torchsummary import summary
summary(model, (3, 128, 128))  # ถ้า input size 128x128 RGB

# ตรวจสอบ output
x = torch.randn(4, 3, 128, 128).to(device)
y = model(x)
print("Output shape:", y.shape)  # [4, 4]


----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 128, 128]           1,728
       BatchNorm2d-2         [-1, 64, 128, 128]             128
              ReLU-3         [-1, 64, 128, 128]               0
            Conv2d-4          [-1, 128, 64, 64]          73,728
       BatchNorm2d-5          [-1, 128, 64, 64]             256
              ReLU-6          [-1, 128, 64, 64]               0
            Conv2d-7          [-1, 128, 64, 64]         147,456
       BatchNorm2d-8          [-1, 128, 64, 64]             256
              ReLU-9          [-1, 128, 64, 64]               0
           Conv2d-10          [-1, 128, 64, 64]         147,456
      BatchNorm2d-11          [-1, 128, 64, 64]             256
             ReLU-12          [-1, 128, 64, 64]               0
       BasicBlock-13          [-1, 128, 64, 64]               0
           Conv2d-14          [-1, 256,

In [ ]:
# ========= Train (no valid set) =========
EPOCHS = 30
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-3)

use_amp = torch.cuda.is_available()
scaler = torch.amp.GradScaler("cuda", enabled=use_amp)

for ep in range(1, EPOCHS+1):
    model.train()
    total, correct, run_loss = 0, 0, 0.0
    for x, y in train_ld:
        x, y = x.to(device, non_blocking=True), y.to(device, non_blocking=True)
        optimizer.zero_grad(set_to_none=True)

        #  ใช้ torch.amp.autocast
        with torch.amp.autocast("cuda", enabled=use_amp):
            logits = model(x)
            loss = criterion(logits, y)

        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()

        run_loss += loss.item() * x.size(0)
        correct  += (logits.argmax(1) == y).sum().item()
        total    += x.size(0)

    print(f"Epoch {ep:02d}/{EPOCHS}  loss {run_loss/total:.4f}  acc {correct/total:.3f}")


In [ ]:

# ========= Test score + Confusion Matrix =========
model.eval()
y_true, y_pred = [], []
with torch.inference_mode(), torch.cuda.amp.autocast(enabled=use_amp):
    for x, y in test_ld:
        x = x.to(device, non_blocking=True)
        logits = model(x)
        y_true.extend(y.numpy().tolist())
        y_pred.extend(logits.argmax(1).cpu().numpy().tolist())

acc = accuracy_score(y_true, y_pred)
print(f"\n=== TEST SCORE ===\nAccuracy: {acc:.4f}")
cm = confusion_matrix(y_true, y_pred, labels=list(range(num_classes)))

fig, ax = plt.subplots(figsize=(6,5), dpi=150)
ConfusionMatrixDisplay(cm, display_labels=[str(r) for r in idx2raw]).plot(ax=ax, cmap="Blues", values_format="d", colorbar=False)
plt.title("Confusion Matrix (counts)"); plt.tight_layout()
plt.setp(ax.get_xticklabels(), rotation=90); plt.show()

cm_norm = cm.astype(float) / cm.sum(axis=1, keepdims=True)
fig, ax = plt.subplots(figsize=(6,5), dpi=150)
ConfusionMatrixDisplay(cm_norm, display_labels=[str(r) for r in idx2raw]).plot(ax=ax, cmap="Blues", values_format=".2f", colorbar=True)
plt.title("Confusion Matrix (row-normalized)"); plt.tight_layout()
plt.setp(ax.get_xticklabels(), rotation=90); plt.show()
